Inspired from 
https://github.com/fchollet/keras/blob/master/examples/addition_rnn.py

Here I am trying to achive sequence to sequence learning. In this case we are trying to enter 2 numbers as text and add them together but with machine learning.

In [ ]:
from keras.models import Sequential
from keras.engine.training import slice_X
from keras.layers import Activation,TimeDistributed,Dense,RepeatVector,recurrent
import numpy as np
from six.moves import range
from keras.utils.visualize_util import plot

In [25]:
class CharacterTable(object):
    def __init__(self,chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c,i) for i,c in enumerate(self.chars))
        self.indices_char = dict((i,c) for i,c in enumerate(self.chars))
        
    def enconde(self,C,num_rows):
        X = np.zeros((num_rows,len(self.chars)))
        for i,c in enumerate(C):
            X[i,self.char_indices[c]] = 1
        return X
    
    def decode(self,X,calc_argmax=True):
        if calc_argmax:
            X = X.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in X)
        
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [26]:
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True
MAXLEN = DIGITS + 1 + DIGITS

chars = '0123456789+ '
ctable = CharacterTable(chars)

In [37]:
questions = []
expected = []
seen = set()
print ("Generating...")
count = 1
while len(questions) < TRAINING_SIZE:
    if count % 10000 == 0:
        print (count)
        
    f = lambda:int(''.join(np.random.choice(list('0123456789'))
                          for i in range(np.random.randint(1,DIGITS+1))))
    a , b = f(), f()
    
    key = tuple(sorted((a,b)))
    if key in seen:
        continue
    seen.add(key)
    
    q = '{}+{}'.format(a,b)
    query = q+' '*(MAXLEN-len(q))
    ans = str(a+b)
    ans +=' '*(DIGITS+1-len(ans))
    
    if INVERT:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
    count += 1
    
    
print ("Total addition questions = {}".format(len(questions)))

    

Generating...
1000
1000
2000
3000
3000
3000
4000
5000
6000
7000
8000
8000
9000
9000
9000
10000
10000
11000
11000
12000
12000
13000
14000
14000
15000
15000
15000
16000
17000
18000
19000
20000
20000
21000
22000
23000
23000
23000
24000
25000
25000
25000
26000
26000
26000
26000
26000
26000
26000
27000
27000
28000
28000
29000
29000
30000
31000
31000
32000
32000
32000
32000
32000
32000
32000
32000
32000
33000
33000
33000
34000
34000
34000
34000
35000
35000
35000
36000
37000
38000
38000
38000
38000
38000
39000
39000
39000
39000
40000
41000
42000
42000
42000
43000
43000
43000
44000
44000
44000
44000
44000
45000
45000
46000
46000
46000
47000
47000
47000
48000
48000
49000
49000
50000
50000
50000
50000
Total addition questions = 50000


In [38]:
# Remember the questions are inverted
print ('Few examples of input and output :')
for i in range(6):
    print ("{} = {}".format(questions[i],expected[i]))


Few examples of input and output :
 827+03 = 758 
  01+53 = 45  
   0+28 = 82  
   0+97 = 79  
 091+96 = 259 
  6+967 = 775 


In [39]:
print ('Vecotrization...')
X = np.zeros((len(questions),MAXLEN,len(chars)),dtype=np.bool)
y = np.zeros((len(questions),DIGITS+1,len(chars)),dtype=np.bool)
for i, sentence in enumerate(questions):
    X[i] = ctable.enconde(sentence,MAXLEN)

for i, sentence in enumerate(expected):
    y[i] = ctable.enconde(sentence,DIGITS+1)
    


Vecotrization...


In [40]:
#Randomizing the input and output
indices = np.arange(len(y))
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

In [41]:
# Separating 10% of the data for test and 90% data for training.
split_at = len(X) - len(X)//10
(X_train, X_test) = (slice_X(X,0,split_at),slice_X(X,split_at))
(y_train, y_test) = (y[:split_at],y[split_at:])

print ("Training Data :")
print (X_train.shape)
print (y_train.shape)

print ("Test Data :")
print (X_test.shape)
print (y_test.shape)

Training Data :
(45000, 7, 12)
(45000, 4, 12)
Test Data :
(5000, 7, 12)
(5000, 4, 12)


In [42]:
RNN = recurrent.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [43]:
model = Sequential()
model.add(RNN(HIDDEN_SIZE,input_shape=(MAXLEN,len(chars))))
model.add(RepeatVector(DIGITS+1))

for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE,return_sequences=True))
    
model.add(TimeDistributed(Dense(len(chars))))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy',
             optimizer='adam', metrics=['accuracy'])

In [44]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_7 (LSTM)                    (None, 128)           72192       lstm_input_4[0][0]               
____________________________________________________________________________________________________
repeatvector_4 (RepeatVector)    (None, 4, 128)        0           lstm_7[0][0]                     
____________________________________________________________________________________________________
lstm_8 (LSTM)                    (None, 4, 128)        131584      repeatvector_4[0][0]             
____________________________________________________________________________________________________
timedistributed_4 (TimeDistribut (None, 4, 12)         1548        lstm_8[0][0]                     
___________________________________________________________________________________________

In [ ]:
for iteration in range(1,100):
    print ()
    print ('-'*50)
    print ('Iteration - ',iteration)
    model.fit(X_train,y_train,batch_size=BATCH_SIZE,nb_epoch=1,validation_data=(X_test,y_test))
    
    for i in range(2):
        ind = np.random.randint(0,len(X_test))
        rowX, rowy = X_test[np.array([ind])], y_test[np.array([ind])]
        preds = model.predict_classes(rowX,verbose=0)
        q = ctable.decode(rowX[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0],calc_argmax=False)
        
        print (" Q ", q[::-1] if INVERT else q)
        print (" T ", correct)
        if correct == guess:
            print (colors.ok +'☑' + colors.close,end = " ")
        else:
            print (colors.fail + '☒' + colors.close,end = " ")
            
        print (guess)
        print ('---')
                
    


--------------------------------------------------
Iteration -  1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 43s - loss: 0.1022 - acc: 0.9811 - val_loss: 0.1040 - val_acc: 0.9754  256/45000 [..............................] - ETA: 45s - loss: 0.1093 - acc: 0.9785

In [12]:
plot(model,to_file='model.png',show_shapes=True)